In [1]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

In [2]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = "./chromedriver.exe" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= "./parse/" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36" # ваш user-agent, узнать его можно тут: https://юзерагент.рф, смотреть через браузер Chrome
start_time = time() # время начала выполнения программы

In [3]:
def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [4]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url", 'comment']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [5]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [6]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
   
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    td_blocks = soup.find_all("td", class_="subtext")
    article = 0
    for i in range(len(tr_blocks)):
        article_id = tr_blocks[i].get("id") # id
        article_url = tr_blocks[i].find_all("a")[1]["href"]
        article_comment = td_blocks[i].find_all("a")[-1].text
        if 'comments' in article_comment:
            article_comment = int(article_comment.split()[0])
        else:
            article_comment = 0
        
        # print(article_comment)

        # иногда статья располагается не на внешнем сайте, а на ycombinator
        # тогда article_url у нее не полный, а добавочный, с параметрами.
        # например item?id=200933. Для этих случаев будем добавлять url до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"
           
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr_blocks[i].span.string,
            "points": score,
            "title": tr_blocks[i].find(class_="titleline").string,
            "url": article_url,
            'comment': article_comment,
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [7]:
start_time = time() # время начала выполнения программы

# инициализируем веб драйвер
browser = webdriver.Chrome(
    service=ChromeService(executable_path=driver_path)
)

# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")

# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

getting page 0...
getting page 1...
getting page 2...
HTTPSConnectionPool(host='phys.org', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.bbc.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='phys.org', port=443): Max retries exceeded with url: /news/2025-05-successful-quantum-error-qudits.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001ED276A6C10>, 'Connection to phys.org timed out. (connect timeout=3.0)'))
HTTPSConnectionPool(host='www.youtube.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='cutfoldtemplates.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='btxx.org', port=443): Max retries exceeded with url: /posts/mail/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001ED276BA2D0>, 'Connection to btxx.org timed out. (connect timeout=3.0)'))
getting page 3...
HTTPSConnectionPool(host='medium.com

In [8]:
from concurrent.futures import ThreadPoolExecutor, wait

# Обернём процедуру парсинга страницы в функцию
def run_process(page_number, filename):
    browser = webdriver.Chrome(
        service=ChromeService(executable_path=driver_path)
    )
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)
       
        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()
       
# Глобальные переменные        
filename = "articles_info_new.csv" # имя файла, в который будем сохранять результат
driver_path = "./chromedriver.exe" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= "./parse/" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"

# Засечём время выполнения кода
start_time = time()

futures = []

# Запустим процесс парсинга на нескольких потоках одновременно
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, number, filename)
        )
       
wait(futures)
end_time = time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

HTTPSConnectionPool(host='www.youtube.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='twitter.com', port=443): Read timed out. (read timeout=3.0)
'NoneType' object has no attribute 'string'
HTTPSConnectionPool(host='apod.nasa.gov', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='sci-hub.se', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.bbc.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='engineering.fb.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.youtube.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.youtube.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='public.support.unisys.com', port=443): M

In [12]:
import pandas as pd

articles_data = pd.read_csv(
    './parse/articles_info.csv',
    names=["id", "load_time", "rank", "points", "title", "url", 'comment'],
    encoding='cp1252'
)

articles_data

,id,load_time,rank,points,title,url,comment
0,44026799,0.542829,1.0,78 points,NaN,https://github.com/aspizu/goboscript,22
1,44020832,0.114064,2.0,83 points,NaN,https://techcrunch.com/2025/05/12/inventwood-i...,69
2,44012283,0.534651,3.0,66 points,NaN,https://www.neowin.net/news/this-printer-compa...,25
3,43995067,0.873323,4.0,171 points,NaN,https://phys.org/news/2025-05-reveals-stronges...,83
4,44027379,0.322102,5.0,12 points,NaN,https://www.awanderingmind.blog/posts/2025-05-...,8
...,...,...,...,...,...,...,...
715,43971944,0.748661,266.0,30 points,NaN,https://arstechnica.com/space/2025/05/tuesday-...,15
716,44010705,0.184925,267.0,113 points,NaN,https://cacm.acm.org/news/the-collapse-of-gpt/,143
717,43995005,0.130785,268.0,260 points,NaN,https://eoinmurray.info/boltzmann-machine,47
718,44016265,0.486395,269.0,45 points,NaN,https://blog.racket-lang.org/2025/05/racket-v8...,3
